<a href="https://colab.research.google.com/github/echeadle/Autogen-colab/blob/main/UdemyAutogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyautogen matplotlib

In [ ]:
from google.colab import userdata
from autogen import ConversableAgent


In [ ]:
llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.8
}

In [ ]:
audiance = ConversableAgent(
    name="audiance",
    llm_config = llm_config,
    is_termination_msg= lambda msg: 'HAHA' in msg['content'].upper(),
    system_message='You are a member of the audiance of a comedy show that is hard to impress.'
)

In [ ]:
comedian = ConversableAgent(
    name="comedian",
    llm_config = llm_config,
    max_consecutive_auto_reply=2,
    system_message='You are a comedian that tells bad jokes'
)

In [ ]:
comedian.initiate_chat(
    audiance,
    message="Welcome to my standup comedy show! Are you ready for a night full of laughter?",
    max_turns = 2
)

comedian (to audiance):

Welcome to my standup comedy show! Are you ready for a night full of laughter?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audiance (to comedian):

We'll see about that. Impress us!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
comedian (to audiance):

Alright, here we go! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audiance (to comedian):

(chuckles) Not bad, not bad. Keep 'em coming!

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Welcome to my standup comedy show! Are you ready for a night full of laughter?', 'role': 'assistant'}, {'content': "We'll see about that. Impress us!", 'role': 'user'}, {'content': 'Alright, here we go! Why did the scarecrow win an award? Because he was outstanding in his field!', 'role': 'assistant'}, {'content': "(chuckles) Not bad, not bad. Keep 'em coming!", 'role': 'user'}], summary="(chuckles) Not bad, not bad. Keep 'em coming!", cost={'usage_including_cached_inference': {'total_cost': 0.000161, 'gpt-3.5-turbo-0125': {'cost': 0.000161, 'prompt_tokens': 184, 'completion_tokens': 46, 'total_tokens': 230}}, 'usage_excluding_cached_inference': {'total_cost': 0.000161, 'gpt-3.5-turbo-0125': {'cost': 0.000161, 'prompt_tokens': 184, 'completion_tokens': 46, 'total_tokens': 230}}}, human_input=[])

In [ ]:
from google.colab import userdata
from autogen import ConversableAgent

In [ ]:
llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.8
}

In [ ]:
agent_guesser = ConversableAgent(
    name="agent_guesser",
    system_message="""Let's play a game. I have a person in mind, and you have to guess it.
    I'll respond with 'yes' or 'no' with a hint. Don't ask questions just make best use of
    information you already have received and return person's name. Go ahead and start guessing!""",
    max_consecutive_auto_reply=1,
    llm_config=llm_config,
    human_input_mode="TERMENATE",
)

agent_thinker = ConversableAgent(
    name="agent_thinker",
    system_message="""Let's play a game.  Think of a famous personality, and I'll try to guess
    their name on every turn. Respond with 'yes' or 'no'. If no, also provide a hint. Let's get started""",
    llm_config=llm_config,
    human_input_mode="NEVER",

)

In [ ]:
result = agent_thinker.initiate_chat(
    agent_guesser,
    message="Let's start. I have someone in mind."
)

agent_thinker (to agent_guesser):

Let's start. I have someone in mind.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_thinker):

Is the person you have in mind a fictional character?

--------------------------------------------------------------------------------
agent_thinker (to agent_guesser):

No

--------------------------------------------------------------------------------
Please give feedback to agent_thinker. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_thinker):

Is the person you have in mind a historical figure?

--------------------------------------------------------------------------------
agent_thinker (to agent_guesser):

Yes

--------------------------------------------------------------------------------
Please give feedback to agent_thinker. Press ente

In [5]:
from google.colab import userdata

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir='code/',
)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)
message_with_code_block = """
Here's a code that solves your problem:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a DataFrame with sample data
data = pd.DataFrame({
  'x': np.arange(10),
  'y': np.random.randint(0, 10, 10)
})

# Plot the line plot
plt.plot(data['x'], data['y'])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Line Plot')
plt.savefig('lineplot.png')
plt.show()
print('Line plot saved to lineplot.png')
```
"""
reply = code_executor_agent.generate_reply(messages=[{"role": "user",
                                                     "content": message_with_code_block}])
print(reply)




>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Figure(640x480)
Line plot saved to lineplot.png



In [6]:
code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks user you coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block)
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web,
2. When you need to perform some task with code, user the code to perform the task and output the result. Finish the
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step
when using code, you must indicate the script type in the code block. The user connot provide any other feedback or
If you want the  user to save the code in a file before executing it, put # filename: <filename> inside the code block
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

llm_config= {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

code_assistant = ConversableAgent(
    "code_assistant",
    llm_config=llm_config,
    system_message=code_assistant_system_message,
    code_execution_config=False
)

In [7]:
chat_result = code_executor_agent.initiate_chat(
    code_assistant,
    message="Write Python code to calculate the 23rd prime number",
    max_turns=2
)

code_executor_agent (to code_assistant):

Write Python code to calculate the 23rd prime number

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_assistant (to code_executor_agent):

To calculate the 23rd prime number, we can write a function to check if a number is prime and then iterate through numbers to find the 23rd prime number.

Here is the Python code:

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

count = 0
num = 2

while True:
    if is_prime(num):
        count += 1
        if count == 23:
            print(num)
            break
    num += 1
```

This code defines a function `is_prime` to check if a number is prime, and then it iterates through numbers starting from 2 to find the 23rd prime number.

You can run this code to calculate the 23rd prime number. Let me know if you need an